In [1]:
import os 
import sys
import subprocess
import pyspark.sql.column
import pyspark.sql.functions as f
from pyspark.sql import SparkSession
from pyspark import StorageLevel
from pyspark.sql.window import Window
from pyspark.sql.types import *
import pandas as pd
import numpy as np
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime, timedelta
import time
import re
import logging
from typing import Callable, List, Dict
import subprocess, re
from subprocess import Popen, PIPE
from datetime import date


from db_connectors import *

print('*********************************')
print('*    ranger_activ_users.py      *')
print('*********************************')

*********************************
*    ranger_activ_users.py      *
*********************************


In [2]:
def get_query_from_df(schema, table, df):
    query_list = []
    df = df.fillna('NULL')
    for index, row in df.iterrows():
        row = row.apply(lambda x: x.replace("'",'`') if type(x) == str else 
                        (str(x) if type(x) == pd._libs.tslibs.timestamps.Timestamp or type(x) == date else 
                         float(x)))
        query_list.append(str(tuple(row)).replace("'NULL'", "NULL"))

    query_full = f'INSERT INTO {schema}.{table} VALUES' +  ', '.join(query_list)
    return query_full

username = "rbsdq_odpp@ROSBANK.RUS.SOCGEN"
p = subprocess.Popen(f"/usr/bin/id {username} -u", shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
id_str1 = p.stdout.readline().decode("utf-8").replace("\n", "")
p = subprocess.Popen(f"id  -u", shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
id_str2 = p.stdout.readline().decode("utf-8").replace("\n", "")
os.environ ["JAVA_TOOL_OPTIONS"] = "-Djava.security.krb5.conf=/etc/krb5.conf.d/krb5.conf"
os.environ ["KRB5_CONFIG"] = "/etc/krb5.conf.d/krb5.conf"
os.environ ["KRB5CCNAME"] = f"/tmp/krb5cc_{id_str1}_{id_str2}"
spark_home = "/usr/hdp/current/spark2-client/"
os.environ ['SPARK_HOME'] = spark_home
sys.path.insert(0, os.path.join(spark_home, 'python'))
os.environ['PYSPARK_PYTHON'] = './environment/bin/python3.7'
os.environ ['HADOOP_CONF_DIR'] = '/usr/hdp/current/hadoop-client/conf/'
os.environ ['PYTHONPATH'] = '/usr/hdp/current/spark2-client/python/lib/py4j-0.10.7-src.zip:/usr/hdp/current/spark2-client/python/'
os.environ ['PYTHONSTARTUP'] = '/usr/hdp/current/spark2-client/python/pyspark/shell.py'
os.environ['PYSPARK_SUBMIT_ARGS'] = '''
--master yarn 
--deploy-mode client 
--num-executors 30
--executor-cores 1 
--driver-memory 12g  
--conf spark.driver.maxResultSize=12g 
--conf spark.serializer=org.apache.spark.serializer.KryoSerializer
--conf spark.sql.execution.arrow.pyspark.enabled=true
--conf spark.sql.crossJoin.enabled=true
--executor-memory 6g 
--conf spark.driver.cores=1
--conf spark.memory.fraction=0.7
--conf spark.shuffle.service.enabled=true 
--conf spark.shuffle.partitions=120
--conf spark.executor.memoryOverhead=2048
--conf spark.driver.memoryOverhead=2048
--conf spark.executor.extraJavaOptions=-XX:+UseG1GC
--conf spark.yarn.dist.archives=hdfs:///datalab/python/python-3.7-dev/python3-venv.tar.gz#environment 
pyspark-shell'''


In [3]:
def listdir(path):
    files = str(subprocess.check_output('hdfs dfs -ls ' + path, shell=True))
    return files.split("\\n")

listdir('hdfs://odpp/ranger/audit/')

["b'Found 6 items",
 'drwx------+  - hbase hbase          0 2022-08-29 00:00 hdfs://odpp/ranger/audit/hbaseMaster',
 'drwx------+  - hbase hbase          0 2022-08-29 00:00 hdfs://odpp/ranger/audit/hbaseRegional',
 'drwxr-x---+  - hdfs  hdfs           0 2022-08-29 00:00 hdfs://odpp/ranger/audit/hdfs',
 'drwx------+  - hive  hive           0 2022-08-29 00:00 hdfs://odpp/ranger/audit/hiveServer2',
 'drwx------+  - kafka kafka          0 2022-08-29 00:00 hdfs://odpp/ranger/audit/kafka',
 'drwx------+  - yarn  yarn           0 2022-08-29 00:00 hdfs://odpp/ranger/audit/yarn',
 "'"]

In [4]:
listdir('hdfs://odpp/ranger/audit/hbaseMaster/')

CalledProcessError: Command 'hdfs dfs -ls hdfs://odpp/ranger/audit/hbaseMaster/' returned non-zero exit status 1.

In [5]:
listdir('hdfs://odpp/ranger/audit/hbaseRegional/')

CalledProcessError: Command 'hdfs dfs -ls hdfs://odpp/ranger/audit/hbaseRegional/' returned non-zero exit status 1.

In [6]:
listdir('hdfs://odpp/ranger/audit/hdfs/')

["b'Found 1199 items",
 'drwxr-xr-x+  - hdfs hdfs          0 2021-11-12 17:00 hdfs://odpp/ranger/audit/hdfs/20190106',
 'drwxr-xr-x+  - hdfs hdfs          0 2021-11-12 17:00 hdfs://odpp/ranger/audit/hdfs/20190109',
 'drwxr-xr-x+  - hdfs hdfs          0 2021-11-12 17:00 hdfs://odpp/ranger/audit/hdfs/20190110',
 'drwxr-xr-x+  - hdfs hdfs          0 2021-11-12 17:00 hdfs://odpp/ranger/audit/hdfs/20190111',
 'drwxr-xr-x+  - hdfs hdfs          0 2021-11-12 17:00 hdfs://odpp/ranger/audit/hdfs/20190112',
 'drwxr-xr-x+  - hdfs hdfs          0 2021-11-12 17:00 hdfs://odpp/ranger/audit/hdfs/20190113',
 'drwxr-xr-x+  - hdfs hdfs          0 2021-11-12 17:00 hdfs://odpp/ranger/audit/hdfs/20190114',
 'drwxr-xr-x+  - hdfs hdfs          0 2021-11-12 17:00 hdfs://odpp/ranger/audit/hdfs/20190115',
 'drwxr-xr-x+  - hdfs hdfs          0 2021-11-12 17:00 hdfs://odpp/ranger/audit/hdfs/20190116',
 'drwxr-xr-x+  - hdfs hdfs          0 2021-11-12 17:00 hdfs://odpp/ranger/audit/hdfs/20190117',
 'drwxr-xr-x+  - 

In [7]:
listdir('hdfs://odpp/ranger/audit/hiveServer2/')

CalledProcessError: Command 'hdfs dfs -ls hdfs://odpp/ranger/audit/hiveServer2/' returned non-zero exit status 1.

In [8]:
listdir('hdfs://odpp/ranger/audit/kafka/')

CalledProcessError: Command 'hdfs dfs -ls hdfs://odpp/ranger/audit/kafka/' returned non-zero exit status 1.

In [9]:
listdir('hdfs://odpp/ranger/audit/yarn/')

CalledProcessError: Command 'hdfs dfs -ls hdfs://odpp/ranger/audit/yarn/' returned non-zero exit status 1.